In [161]:
from  pyspark.sql.functions import input_file_name
from pyspark.sql.functions  import regexp_extract
from pyspark.sql.functions import regexp_extract, col, split, size
from pyspark.sql import *
from pyspark.conf import SparkConf
SparkSession.builder.config(conf=SparkConf())
# A=spark.read.options(header='True',inferSchema='True',delimiter=',').csv("A.txt")
# b=A.withColumn("filename", input_file_name())
# b.show(10, False)

+----------+------+------+-------+------+-------+-------------------------------------+
|timestamp |open  |high  |low    |close |volume |filename                             |
+----------+------+------+-------+------+-------+-------------------------------------+
|2022-03-15|129.22|131.15|128.865|130.28|1649205|gs://terradataproc-bucket-53533/A.txt|
|2022-03-14|131.0 |132.26|126.9  |127.58|1737149|gs://terradataproc-bucket-53533/A.txt|
|2022-03-11|135.43|136.44|130.16 |130.31|1815769|gs://terradataproc-bucket-53533/A.txt|
|2022-03-10|131.4 |134.52|131.29 |134.33|1933127|gs://terradataproc-bucket-53533/A.txt|
|2022-03-09|132.85|133.94|131.19 |133.26|1805585|gs://terradataproc-bucket-53533/A.txt|
|2022-03-08|128.64|132.5 |127.54 |129.32|2375483|gs://terradataproc-bucket-53533/A.txt|
|2022-03-07|133.55|133.55|128.43 |130.34|2376504|gs://terradataproc-bucket-53533/A.txt|
|2022-03-04|135.85|137.12|132.14 |133.89|3848511|gs://terradataproc-bucket-53533/A.txt|
|2022-03-03|133.73|138.0 |133.44

In [125]:
#Extract file name as column
def extract_filename(tableDF):
    b=tableDF.withColumn("filename", regexp_extract(input_file_name(), r'(?<=.*\/.*\/.*\/).+', 0))
    c=b.withColumn("Symbol1", regexp_extract("filename", r'^([^.])+', 0))
    #print(c)
    return(c)


In [140]:
#Read diff csv files and craete new column called symbol by passing 
#the df to above function to add filename in the symbol col to join with meta table
#due to huge amount of files, I just took only 2 files but can add all the files sent to the above function
A=spark.read.options(header='True',inferSchema='True',delimiter=',').csv("A.txt")
A=extract_filename(A)
print(A.count())
A.printSchema()

AA=spark.read.options(header='True',inferSchema='True',delimiter=',').csv("AA.txt")
AA=extract_filename(AA)
print(AA.count())
AA.printSchema()

metadata=spark.read.options(header='True',inferSchema='True',delimiter=',').csv("meta_sample.txt")
metadata.printSchema()

#df.withColumn("filename", input_file_name()).show(10, False)

5616
root
 |-- timestamp: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- filename: string (nullable = false)
 |-- Symbol1: string (nullable = false)

1361
root
 |-- timestamp: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- filename: string (nullable = false)
 |-- Symbol1: string (nullable = false)

root
 |-- Symbol: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Address: string (nullable = true)



In [151]:
unoindfs=A.union(AA)
unionplusjoin=unoindfs.join(metadata, unoindfs['Symbol1'] == metadata['Symbol'], "inner")


In [156]:
#report 1 Generate summary reports for all time till yesterday
from pyspark.sql.functions import *
report1=unionplusjoin.groupBy("Sector") \
       .agg(avg("open").alias("Avg_Open_Price"), \
            avg("close").alias("Avg_Close_Price"), \
            max("high").alias("Max_High_Price"), \
            min("low").alias("Min_Low_Price"), \
            avg("volume").alias("Avg_Volume"))
report1.show(20, False)

+-------------+------------------+------------------+--------------+-------------+------------------+
|Sector       |Avg_Open_Price    |Avg_Close_Price   |Max_High_Price|Min_Low_Price|Avg_Volume        |
+-------------+------------------+------------------+--------------+-------------+------------------+
|LIFE SCIENCES|48.928510505698235|48.939718660968595|179.57        |10.5         |2710846.7519586897|
|MANUFACTURING|32.826966348273345|32.84054371785455 |92.32         |5.16         |5396276.952240999 |
+-------------+------------------+------------------+--------------+-------------+------------------+



In [159]:
#report 2 Generate summary reports for the given period of time

filter_days=unionplusjoin.filter("timestamp between '2021-01-01' and '2021-05-26'")

report2=filter_days.groupBy("Sector") \
       .agg(avg("open").alias("Avg_Open_Price"), \
            avg("close").alias("Avg_Close_Price"), \
            max("high").alias("Max_High_Price"), \
            min("low").alias("Min_Low_Price"), \
            avg("volume").alias("Avg_Volume"))
report2.show(20, False)

+-------------+------------------+------------------+--------------+-------------+----------+
|Sector       |Avg_Open_Price    |Avg_Close_Price   |Max_High_Price|Min_Low_Price|Avg_Volume|
+-------------+------------------+------------------+--------------+-------------+----------+
|LIFE SCIENCES|127.09909999999999|127.01569999999998|138.0         |112.47       |1626721.94|
|MANUFACTURING|29.425661000000012|29.48590000000001 |44.42         |17.3         |6822501.93|
+-------------+------------------+------------------+--------------+-------------+----------+



In [160]:
#report 3  Generate detailed reports for the given sector + given period of time

filter_days_report3=unionplusjoin.filter("timestamp between '2021-02-21' and '2021-05-26'")

report3=filter_days_report3.groupBy("Symbol","Name") \
       .agg(avg("open").alias("Avg_Open_Price"), \
            avg("close").alias("Avg_Close_Price"), \
            max("high").alias("Max_High_Price"), \
            min("low").alias("Min_Low_Price"), \
            avg("volume").alias("Avg_Volume"))
report3.show(20, False)

+------+------------------------+------------------+------------------+--------------+-------------+------------------+
|Symbol|Name                    |Avg_Open_Price    |Avg_Close_Price   |Max_High_Price|Min_Low_Price|Avg_Volume        |
+------+------------------------+------------------+------------------+--------------+-------------+------------------+
|A     |Agilent Technologies Inc|128.13925373134327|128.08119402985076|138.0         |112.47       |1569628.7014925373|
|AA    |Alcoa Corporation       |33.151807462686584|33.27567164179105 |44.42         |22.95        |7090108.059701492 |
+------+------------------------+------------------+------------------+--------------+-------------+------------------+



In [163]:
#sample output storage code to write gcs

report3.write.format("csv").mode("append").save("gs://terradataproc-bucket-53533/data.txt")